# Time Series Forecasting with AWS Bedrock Runtime

This notebook demonstrates how to use the AWS Bedrock Runtime client to invoke a time series forecasting model.

**Note:** Ensure you have the necessary AWS credentials configured and the model endpoint deployed in the specified region.

The payloads are structured as per the model's requirements:
- `target`: The historical time series data.
- `past_covariates` (optional): Historical features matching the target's length. Can include numeric and categorical features.
- `future_covariates` (optional): Future features matching the prediction length.
- `parameters`: Includes `prediction_length` (how many steps to forecast) and `quantile_levels` (quantiles for uncertainty estimates).

## Step 1: Import Required Libraries

In [1]:
import boto3
import json

## Step 2: Initialize the Bedrock Runtime Client

Here, we create a client for the Bedrock Runtime service. 
Replace `'us-west-2'` with the region where your model endpoint is deployed.
This client will be used to invoke the model.

In [2]:
client = boto3.client('bedrock-runtime', region_name='us-west-2')

## Step 3: Define the Model ID

The `model_id` is the ARN of your deployed SageMaker endpoint.
Replace the example ARN with your actual one from the AWS console.

In [3]:
model_id = 'arn:aws:sagemaker:us-west-2:609009159737:endpoint/endpoint-quick-start-ov3et'  # Example ARN; update with yours

## Step 4: Prepare Example Payloads

We define two example payloads:

### Basic Payload
- Only includes the `target` time series and forecasting parameters.
- Forecasts 10 steps ahead with quantiles at 0.1, 0.5 (median), and 0.9.

### Payload with Covariates
- Includes `past_covariates` (historical features) and `future_covariates` (future features).
- Covariates can be numeric (e.g., `feat_1`) or categorical (e.g., `feat_2` as strings).
- Lengths: `past_covariates` matches `target`; `future_covariates` matches `prediction_length`.
- Forecasts 3 steps ahead in this example.

In [4]:
# Basic payload without covariates
payload_basic = {
    "inputs": [
        {
            "target": [0.0, 4.0, 5.0, 1.5, -3.0, -5.0, -3.0, 1.5, 5.0, 4.0, 0.0, -4.0, -5.0, -1.5, 3.0, 5.0, 3.0, -1.5, -5.0, -4.0]
        }
    ],
    "parameters": {
        "prediction_length": 10,
        "quantile_levels": [0.1, 0.5, 0.9]
    }
}

In [5]:
# Payload with covariates
payload_with_covariates = {
    "inputs": [
        {
            "target": [0.0, 4.0, 5.0, 1.5, -3.0, -5.0, -3.0, 1.5, 5.0, 4.0],
            "past_covariates": {
                "feat_1": [0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 2.0, 0.0, 3.0, 1.0],  # Numeric feature
                "feat_2": ["A", "A", "A", "B", "B", "A", "B", "C", "A", "A"]   # Categorical feature (strings)
            },
            "future_covariates": {
                "feat_1": [0.5, 1.0, 1.0],
                "feat_2": ["C", "B", "C"]
            }
        }
    ],
    "parameters": {
        "prediction_length": 3,
        "quantile_levels": [0.1, 0.5, 0.9]
    }
}

## Step 5: Select a Payload

Choose which payload to use for the model invocation.
Uncomment the other line to switch to the covariates example.

In [6]:
# Select the basic payload (swap to payload_with_covariates if desired)
payload = payload_basic  # Or: payload = payload_with_covariates

## Step 6: Invoke the Model

We send the payload to the model endpoint using the Bedrock Runtime client.
- `body`: JSON-encoded payload.
- `modelId`: The ARN defined earlier.
- `accept` and `contentType`: Set to JSON for input/output.

In [7]:
response = client.invoke_model(
    body=json.dumps(payload),
    modelId=model_id,
    accept='application/json',
    contentType='application/json'
)

## Step 7: Parse and Print the Response

The response contains the forecasted values.
- We read and decode the response body.
- Parse it as JSON.
- Pretty-print the output, showing mean forecasts and quantiles.

In [8]:
response_body = json.loads(response['body'].read().decode('utf-8'))
print(json.dumps(response_body, indent=4))  # Outputs mean and quantile forecasts

{
    "predictions": [
        {
            "mean": [
                -1.5533066987991333,
                0.562282919883728,
                1.9398761987686157,
                1.4267929792404175,
                -1.033194899559021,
                -3.359640598297119,
                -2.670844078063965,
                -0.6044541597366333,
                1.0121092796325684,
                1.6446776390075684
            ],
            "0.1": [
                -4.132779598236084,
                -2.670844078063965,
                -1.6306205987930298,
                -2.3053600788116455,
                -4.779405117034912,
                -6.97230863571167,
                -6.578710556030273,
                -4.807519435882568,
                -3.3877546787261963,
                -2.839528799057007
            ],
            "0.5": [
                -1.5533066987991333,
                0.562282919883728,
                1.9398761987686157,
                1.4267929792404175,
        